In [1]:
import time
import numpy as np
import pandas as pd
from dateutil.parser import parse
from datetime import date, timedelta
from sklearn.preprocessing import LabelEncoder


data_path = '../data/'

air_reserve = pd.read_csv(data_path + 'air_reserve.csv').rename(columns={'air_store_id':'store_id'})
hpg_reserve = pd.read_csv(data_path + 'hpg_reserve.csv').rename(columns={'hpg_store_id':'store_id'})
air_store = pd.read_csv(data_path + 'air_store_info.csv').rename(columns={'air_store_id':'store_id'})
hpg_store = pd.read_csv(data_path + 'hpg_store_info.csv').rename(columns={'hpg_store_id':'store_id'})
air_visit = pd.read_csv(data_path + 'air_visit_data.csv').rename(columns={'air_store_id':'store_id'})
store_id_map = pd.read_csv(data_path + 'store_id_relation.csv').set_index('hpg_store_id',drop=False)
date_info = pd.read_csv(data_path + 'date_info.csv').rename(columns={'calendar_date': 'visit_date'}).drop('day_of_week',axis=1)
submission = pd.read_csv(data_path + 'sample_submission.csv')

提出用データについて、訪問日付とストアidを分離

In [2]:
submission['visit_date'] = submission['id'].str[-10:]
submission['store_id'] = submission['id'].str[:-11]
submission.head()

,id,visitors,visit_date,store_id
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9


- 訪問日と予約日について、それぞれ日付のみ抽出
- Day of Week カラムの追加(strをdatetimeになおしてから)

In [3]:
air_reserve['visit_date'] = air_reserve['visit_datetime'].str[:10]
air_reserve['reserve_date'] = air_reserve['reserve_datetime'].str[:10]
air_reserve['dow'] = pd.to_datetime(air_reserve['visit_date']).dt.dayofweek
air_reserve.head()

,store_id,visit_datetime,reserve_datetime,reserve_visitors,visit_date,reserve_date,dow
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1,2016-01-01,2016-01-01,4
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3,2016-01-01,2016-01-01,4
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6,2016-01-01,2016-01-01,4
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2,2016-01-01,2016-01-01,4
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5,2016-01-01,2016-01-01,4


- 教師データとして使えるように、store_id と visit_date を統合して submission に合わせた形にする

In [4]:
air_visit['id'] = air_visit['store_id'] + '_' + air_visit['visit_date']
air_visit.head()

,store_id,visit_date,visitors,id
0,air_ba937bf13d40fb24,2016-01-13,25,air_ba937bf13d40fb24_2016-01-13
1,air_ba937bf13d40fb24,2016-01-14,32,air_ba937bf13d40fb24_2016-01-14
2,air_ba937bf13d40fb24,2016-01-15,29,air_ba937bf13d40fb24_2016-01-15
3,air_ba937bf13d40fb24,2016-01-16,22,air_ba937bf13d40fb24_2016-01-16
4,air_ba937bf13d40fb24,2016-01-18,6,air_ba937bf13d40fb24_2016-01-18


In [5]:
hpg_reserve['visit_date'] = hpg_reserve['visit_datetime'].str[:10]
hpg_reserve['reserve_date'] = hpg_reserve['reserve_datetime'].str[:10]
hpg_reserve['dow'] = pd.to_datetime(hpg_reserve['visit_date']).dt.dayofweek

hpg_reserve['store_id'] = hpg_reserve['store_id'].map(store_id_map['air_store_id']).fillna(hpg_reserve['store_id'])
hpg_reserve.head()

hpg_store['store_id'] = hpg_store['store_id'].map(store_id_map['air_store_id']).fillna(hpg_store['store_id'])
hpg_store.rename(columns={'hpg_genre_name':'air_genre_name','hpg_area_name':'air_area_name'},inplace=True)
hpg_store.head()

,store_id,air_genre_name,air_area_name,latitude,longitude
0,hpg_6622b62385aec8bf,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
1,hpg_e9e068dd49c5fa00,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
2,hpg_2976f7acb4b3a3bc,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
3,hpg_e51a522e098f024c,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
4,hpg_e3d0e1519894f275,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221


In [6]:
hpg_reserve.head()

,store_id,visit_datetime,reserve_datetime,reserve_visitors,visit_date,reserve_date,dow
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1,2016-01-01,2016-01-01,4
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3,2016-01-01,2016-01-01,4
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2,2016-01-01,2016-01-01,4
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5,2016-01-01,2016-01-01,4
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13,2016-01-01,2016-01-01,4


- air_visit と submission をconcatすることで、全期間のデータを data に統合。

In [7]:
data = pd.concat([air_visit, submission]).copy()
data['dow'] = pd.to_datetime(data['visit_date']).dt.dayofweek
data.tail()

,store_id,visit_date,visitors,id,dow
32014,air_fff68b929994bfbd,2017-05-27,0,air_fff68b929994bfbd_2017-05-27,5
32015,air_fff68b929994bfbd,2017-05-28,0,air_fff68b929994bfbd_2017-05-28,6
32016,air_fff68b929994bfbd,2017-05-29,0,air_fff68b929994bfbd_2017-05-29,0
32017,air_fff68b929994bfbd,2017-05-30,0,air_fff68b929994bfbd_2017-05-30,1
32018,air_fff68b929994bfbd,2017-05-31,0,air_fff68b929994bfbd_2017-05-31,2


- holiday_flg: The day is holiday.
- holiday_flg2: The day is Suturday or Sunday or holiday.

In [8]:
date_info['data_info_dow'] = pd.to_datetime(date_info['visit_date']).dt.dayofweek#
date_info['holiday_flg2'] = ((date_info['data_info_dow']>4) | (date_info['holiday_flg']==1)).astype(int)
date_info = date_info.drop('data_info_dow', axis=1)
date_info.head()

,visit_date,holiday_flg,holiday_flg2
0,2016-01-01,1,1
1,2016-01-02,1,1
2,2016-01-03,1,1
3,2016-01-04,0,0
4,2016-01-05,0,0


In [9]:
# air_area_name の県名と air_genre_name でラベルエンコーディング

In [10]:
air_store['air_area_name0'] = air_store['air_area_name'].apply(lambda x: x.split(' ')[0])
lbl = LabelEncoder()
air_store['air_genre_name'] = lbl.fit_transform(air_store['air_genre_name'])
air_store['air_area_name0'] = lbl.fit_transform(air_store['air_area_name0'])

In [11]:
air_store.head()

,store_id,air_genre_name,air_area_name,latitude,longitude,air_area_name0
0,air_0f0cdeee6c9bf3d7,6,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,3
1,air_7cc17a324ae5c7dc,6,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,3
2,air_fee8dcf4d619598e,6,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,3
3,air_a17f0778617c76e2,6,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,3
4,air_83db5aff8f50478e,6,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7


- 評価関数がRMLSEなので、前もって訪問者数を対数化しておく
- Since the evaluation function is RMLSE, logarithmize the number of visitors in advance.

In [12]:
data['visitors'] = np.log1p(data['visitors'])
data = data.merge(air_store,on='store_id',how='left')
data = data.merge(date_info[['visit_date','holiday_flg','holiday_flg2']], on=['visit_date'],how='left')
data.head()

,store_id,visit_date,visitors,id,dow,air_genre_name,air_area_name,latitude,longitude,air_area_name0,holiday_flg,holiday_flg2
0,air_ba937bf13d40fb24,2016-01-13,3.258097,air_ba937bf13d40fb24_2016-01-13,2,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7,0,0
1,air_ba937bf13d40fb24,2016-01-14,3.496508,air_ba937bf13d40fb24_2016-01-14,3,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7,0,0
2,air_ba937bf13d40fb24,2016-01-15,3.401197,air_ba937bf13d40fb24_2016-01-15,4,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7,0,0
3,air_ba937bf13d40fb24,2016-01-16,3.135494,air_ba937bf13d40fb24_2016-01-16,5,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7,0,1
4,air_ba937bf13d40fb24,2016-01-18,1.945910,air_ba937bf13d40fb24_2016-01-18,0,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7,0,0


In [13]:
def date_add_days(start_date, days):
    """ Return date after a specified date from the start date.
    Args:
        start_date (str): The start date
        days (int): Number of days to be added to the start date
    Return:
        
    """
    end_date = parse(start_date[:10]) + timedelta(days=days)
    end_date = end_date.strftime('%Y-%m-%d')
    return end_date

In [14]:
def concat(L):
    result = None
    for l in L:
        if result is None:
            result = l
        else:
            try:
                result[l.columns.tolist()] = l
            except:
                print(l.head())
    return result

def left_merge(data1,data2,on):
    if type(on) != list:
        on = [on]
    if (set(on) & set(data2.columns)) != set(on):
        data2_temp = data2.reset_index()
    else:
        data2_temp = data2.copy()
    columns = [f for f in data2.columns if f not in on]
    result = data1.merge(data2_temp,on=on,how='left')
    result = result[columns]
    return result

In [15]:
def diff_of_days(day1, day2):
    """ Parse the given date and calculate diff of days.
    Args:
        day1, day2 (str): date of the day.
    Returns:
        Number of diff days between day1 and day2.
    """
    days = (parse(day1[:10]) - parse(day2[:10])).days
    return days

In [16]:
def date_add_days(start_date, days):
    """ Parse the given start_date and add the days."""
    end_date = parse(start_date[:10]) + timedelta(days=days)
    end_date = end_date.strftime('%Y-%m-%d')
    return end_date

In [17]:
def get_label(end_date,n_day):
    """ Get Label whith has ...
    """
    # (学習期間の？)end_date に n_day 足してる、教師データにする用？
    label_end_date = date_add_days(end_date, n_day)
    
    # visit_date が label_end_date よりも小さくて、 end_date よりも大きい範囲をコピー
    label = data[(data['visit_date'] < label_end_date) & (data['visit_date'] >= end_date)].copy()
    
    # visit_date から何日離れているかを算出
    label['end_date'] = end_date
    label['diff_of_day'] = label['visit_date'].apply(lambda x: diff_of_days(x,end_date))
        
    label['month'] = label['visit_date'].str[5:7].astype(int)
    label['year'] = label['visit_date'].str[:4].astype(int)
    
    
    for i in [3,2,1,-1]:
        date_info_temp = date_info.copy()
        
        # visit_date について 3、２、1、-1日前の date を確保
        date_info_temp['visit_date'] = date_info_temp['visit_date'].apply(lambda x: date_add_days(x,i))
        
        # その後、3、２、1、-1日前の holiday_flg を特徴量に追加
        date_info_temp.rename(columns={'holiday_flg':'ahead_holiday_{}'.format(i),'holiday_flg2':'ahead_holiday2_{}'.format(i)},inplace=True)
        label = label.merge(date_info_temp, on=['visit_date'],how='left')
    
    label = label.reset_index(drop=True)
    return label, label_end_date

In [18]:
def get_store_visitor_feat(label, key, n_day):
    """ Get store visitor features.
    Args:
        label: 
        key (tupple): end_date(str), n_day(int)
        n_day: 
    """
    start_date = date_add_days(key[0],-n_day)
    
    # 必要な時期のデータのみを抽出
    data_temp = data[(data.visit_date < key[0]) & (data.visit_date > start_date)].copy()
    
    # store_id で groupby した上で、
    store_data = data_temp.groupby(['store_id'],as_index=False)
    
    # visitors の統計量を算出
    result = store_data['visitors'].agg(
            {'store_min{}'.format(n_day): 'min',
             'store_mean{}'.format(n_day): 'mean',
             'store_median{}'.format(n_day): 'median',
             'store_max{}'.format(n_day): 'max',
             'store_count{}'.format(n_day): 'count',
             'store_std{}'.format(n_day): 'std',
             'store_skew{}'.format(n_day): 'skew'}
    )
    result = left_merge(label, result, on=['store_id']).fillna(0)
    return result

In [19]:
def get_store_exp_visitor_feat(label, key, n_day):
    """ Get store expected visit feature as weighted mean(sum of weighted visitors)/(sum of weights).
    Args:
        label: 
        key (tupple): end_date(str), n_day(int)
        n_day: 
    Returns:
        result(pd.DataFrame)
    """
    start_date = date_add_days(key[0], -n_day)
    data_temp = data[(data.visit_date < key[0]) & (data.visit_date > start_date)].copy()
    
    data_temp['visit_date'] = data_temp['visit_date'].apply(lambda x: diff_of_days(key[0],x))
    data_temp['weight'] = data_temp['visit_date'].apply(lambda x: 0.985**x)
    
    data_temp['visitors'] = data_temp['visitors'] * data_temp['weight']
    
    result1 = data_temp.groupby(['store_id'], as_index=False)['visitors'].agg({'store_exp_mean{}'.format(n_day): 'sum'})
    result2 = data_temp.groupby(['store_id'], as_index=False)['weight'].agg({'store_exp_weight_sum{}'.format(n_day): 'sum'})
    
    result = result1.merge(result2, on=['store_id'], how='left')
    result['store_exp_mean{}'.format(n_day)] = result['store_exp_mean{}'.format(n_day)]/result['store_exp_weight_sum{}'.format(n_day)]
    result = left_merge(label, result, on=['store_id']).fillna(0)
    return result

In [20]:
def get_store_week_feat(label, key, n_day):
    """ Get store visitor features for each day of the week.
    Args:
        label: 
        key (tupple): end_date(str), n_day(int)
        n_day: 
    Returns:
        result (pd.DataFrame)
    """
    start_date = date_add_days(key[0], -n_day)
    data_temp = data[(data.visit_date < key[0]) & (data.visit_date > start_date)].copy()
    store_dow = data_temp.groupby(['store_id', 'dow'], as_index=False)
    result = store_dow['visitors'].agg(
        {'store_dow_min{}'.format(n_day): 'min',
         'store_dow_mean{}'.format(n_day): 'mean',
         'store_dow_median{}'.format(n_day): 'median',
         'store_dow_max{}'.format(n_day): 'max',
         'store_dow_count{}'.format(n_day): 'count',
         'store_dow_std{}'.format(n_day): 'std',
         'store_dow_skew{}'.format(n_day): 'skew'}
    )
    result = left_merge(label, result, on=['store_id', 'dow']).fillna(0)
    return result

In [21]:
def get_store_week_diff_feat(label, key, n_day):
    """ Get difference in the number of visitors for each day and calcurate key statistics.
    Args:
        label: 
        key (tupple): end_date(str), n_day(int)
        n_day:
    Returns:
        result (pd.DataFrame)
    """  
    start_date = date_add_days(key[0], -n_day)
    data_temp = data[(data.visit_date < key[0]) & (data.visit_date > start_date)].copy()
    
    result = data_temp.set_index(['store_id','visit_date'])['visitors'].unstack()
    result = result.diff(axis=1).iloc[:,1:]
    c = result.columns
    
    result['store_diff_mean'] = np.abs(result[c]).mean(axis=1)
    result['store_diff_std'] = result[c].std(axis=1)
    result['store_diff_max'] = result[c].max(axis=1)
    result['store_diff_min'] = result[c].min(axis=1)
    
    cols = ['store_diff_mean', 'store_diff_std', 'store_diff_max', 'store_diff_min']
    
    result = left_merge(label, result[cols], on=['store_id']).fillna(0)
    return result

In [22]:
def get_store_all_week_feat(label, key, n_day):
    """ Get store visitor features for each day of the week.???
    Args:
        label: 
        key (tupple): end_date(str), n_day(int)
        n_day:
    Returns:
        result (pd.DataFrame)
    """
    start_date = date_add_days(key[0], -n_day)
    data_temp = data[(data.visit_date < key[0]) & (data.visit_date > start_date)].copy()
    
    store_all_week = data_temp.groupby(['store_id', 'dow'],as_index=False)
    result_temp = store_all_week['visitors'].agg(
        {'store_dow_mean{}'.format(n_day): 'mean',
         'store_dow_median{}'.format(n_day): 'median',
         'store_dow_sum{}'.format(n_day): 'max',
         'store_dow_count{}'.format(n_day): 'count'}
    )
    
    result = pd.DataFrame()
    for i in range(7):
        result_sub = result_temp[result_temp['dow']==i].copy()
        result_sub = result_sub.set_index('store_id')
        result_sub = result_sub.add_prefix(str(i))
        result_sub = left_merge(label, result_sub, on=['store_id']).fillna(0)
        result = pd.concat([result,result_sub],axis=1)
    
    return result

In [23]:
def get_store_week_exp_feat(label, key, n_day):
    """ Get store expected visit feature for each day of the week 
    as weighted mean(sum of weighted visitors)/(sum of weights).
    Args:
        label: 
        key (tupple): end_date(str), n_day(int)
        n_day: 
    Returns:
        result(pd.DataFrame)
    """
    start_date = date_add_days(key[0], -n_day)
    data_temp = data[(data.visit_date < key[0]) & (data.visit_date > start_date)].copy()
    data_temp['visit_date'] = data_temp['visit_date'].apply(lambda x: diff_of_days(key[0],x))
    data_temp['visitors2'] = data_temp['visitors']
    result = None
    for i in [0.9,0.95,0.97,0.98,0.985,0.99,0.999,0.9999]:
        data_temp['weight'] = data_temp['visit_date'].apply(lambda x: i**x)
        data_temp['visitors1'] = data_temp['visitors'] * data_temp['weight']
        data_temp['visitors2'] = data_temp['visitors2'] * data_temp['weight']
        result1 = data_temp.groupby(['store_id', 'dow'], as_index=False)['visitors1'].agg({'store_dow_exp_mean{}_{}'.format(n_day,i): 'sum'})
        result3 = data_temp.groupby(['store_id', 'dow'], as_index=False)['visitors2'].agg({'store_dow_exp_mean2{}_{}'.format(n_day, i): 'sum'})
        result2 = data_temp.groupby(['store_id', 'dow'], as_index=False)['weight'].agg({'store_dow_exp_weight_sum{}_{}'.format(n_day,i): 'sum'})
        result_temp = result1.merge(result2, on=['store_id', 'dow'], how='left')
        result_temp = result_temp.merge(result3, on=['store_id', 'dow'], how='left')
        result_temp['store_dow_exp_mean{}_{}'.format(n_day,i)] = result_temp['store_dow_exp_mean{}_{}'.format(n_day,i)]/result_temp['store_dow_exp_weight_sum{}_{}'.format(n_day,i)]
        result_temp['store_dow_exp_mean2{}_{}'.format(n_day, i)] = result_temp[ 'store_dow_exp_mean2{}_{}'.format(n_day, i)]/result_temp['store_dow_exp_weight_sum{}_{}'.format(n_day, i)]
        if result is None:
            result = result_temp
        else:
            result = result.merge(result_temp,on=['store_id','dow'],how='left')
    result = left_merge(label, result, on=['store_id', 'dow']).fillna(0)
    return result

In [24]:
def get_store_holiday_feat(label, key, n_day):
    """ Get store visitors feature for hoidays.
    Args:
        label: 
        key (tupple): end_date(str), n_day(int)
        n_day: 
    Returns:
        result(pd.DataFrame)
    """
    
    start_date = date_add_days(key[0], -n_day)
    data_temp = data[(data.visit_date < key[0]) & (data.visit_date > start_date)].copy()
    result1 = data_temp.groupby(['store_id', 'holiday_flg'], as_index=False)['visitors'].agg(
        {'store_holiday_min{}'.format(n_day): 'min',
         'store_holiday_mean{}'.format(n_day): 'mean',
         'store_holiday_median{}'.format(n_day): 'median',
         'store_holiday_max{}'.format(n_day): 'max',
         'store_holiday_count{}'.format(n_day): 'count',
         'store_holiday_std{}'.format(n_day): 'std',
         'store_holiday_skew{}'.format(n_day): 'skew'})
    result1 = left_merge(label, result1, on=['store_id', 'holiday_flg']).fillna(0)
    result2 = data_temp.groupby(['store_id', 'holiday_flg2'], as_index=False)['visitors'].agg(
        {'store_holiday2_min{}'.format(n_day): 'min',
         'store_holiday2_mean{}'.format(n_day): 'mean',
         'store_holiday2_median{}'.format(n_day): 'median',
         'store_holiday2_max{}'.format(n_day): 'max',
         'store_holiday2_count{}'.format(n_day): 'count',
         'store_holiday2_std{}'.format(n_day): 'std',
         'store_holiday2_skew{}'.format(n_day): 'skew'})
    result2 = left_merge(label, result2, on=['store_id', 'holiday_flg2']).fillna(0)
    result = pd.concat([result1, result2], axis=1)
    return result

In [25]:
def get_genre_visitor_feat(label, key, n_day):
    """ Get genre visitors features.
    Args:
        label: 
        key (tupple): end_date(str), n_day(int)
        n_day: 
    Returns:
        result(pd.DataFrame)
    """
    start_date = date_add_days(key[0],-n_day)
    data_temp = data[(data.visit_date < key[0]) & (data.visit_date > start_date)].copy()
    air_genre = data_temp.groupby(['air_genre_name'], as_index=False)
    
    result = air_genre['visitors'].agg(
        {'genre_min{}'.format(n_day): 'min',
         'genre_mean{}'.format(n_day): 'mean',
         'genre_median{}'.format(n_day): 'median',
         'genre_max{}'.format(n_day): 'max',
         'genre_count{}'.format(n_day): 'count',
         'genre_std{}'.format(n_day): 'std',
         'genre_skew{}'.format(n_day): 'skew'}
    )
    result = left_merge(label, result, on=['air_genre_name']).fillna(0)
    return result

In [26]:
def get_genre_exp_visitor_feat(label, key, n_day):
    """ Get genre expected visit feature for each day of the week 
    as weighted mean(sum of weighted visitors)/(sum of weights).
    Args:
        label: 
        key (tupple): end_date(str), n_day(int)
        n_day: 
    Returns:
        result(pd.DataFrame)
    """
    start_date = date_add_days(key[0], -n_day)
    data_temp = data[(data.visit_date < key[0]) & (data.visit_date > start_date)].copy()
    data_temp['visit_date'] = data_temp['visit_date'].apply(lambda x: diff_of_days(key[0],x))
    data_temp['weight'] = data_temp['visit_date'].apply(lambda x: 0.985**x)
    data_temp['visitors'] = data_temp['visitors'] * data_temp['weight']
    result1 = data_temp.groupby(['air_genre_name'], as_index=False)['visitors'].agg({'genre_exp_mean{}'.format(n_day): 'sum'})
    result2 = data_temp.groupby(['air_genre_name'], as_index=False)['weight'].agg({'genre_exp_weight_sum{}'.format(n_day): 'sum'})
    result = result1.merge(result2, on=['air_genre_name'], how='left')
    result['genre_exp_mean{}'.format(n_day)] = result['genre_exp_mean{}'.format(n_day)]/result['genre_exp_weight_sum{}'.format(n_day)]
    result = left_merge(label, result, on=['air_genre_name']).fillna(0)
    return result

In [27]:
def get_genre_week_feat(label, key, n_day):
    """ Get genre visitor features for each day of the week.
    Args:
        label: 
        key (tupple): end_date(str), n_day(int)
        n_day: 
    Returns:
        result (pd.DataFrame)
    """
    start_date = date_add_days(key[0], -n_day)
    data_temp = data[(data.visit_date < key[0]) & (data.visit_date > start_date)].copy()
    
    genre_week = data_temp.groupby(['air_genre_name', 'dow'], as_index=False)
    result = genre_week['visitors'].agg({'genre_dow_min{}'.format(n_day): 'min',
                                         'genre_dow_mean{}'.format(n_day): 'mean',
                                         'genre_dow_median{}'.format(n_day): 'median',
                                         'genre_dow_max{}'.format(n_day): 'max',
                                         'genre_dow_count{}'.format(n_day): 'count',
                                         'genre_dow_std{}'.format(n_day): 'std',
                                         'genre_dow_skew{}'.format(n_day): 'skew'})
    result = left_merge(label, result, on=['air_genre_name', 'dow']).fillna(0)
    return result

In [28]:
def get_genre_week_exp_feat(label, key, n_day):
    """ Get genre visitor features for each day of the week.
    Args:
        label: 
        key (tupple): end_date(str), n_day(int)
        n_day: 
    Returns:
        result (pd.DataFrame)
    """
    start_date = date_add_days(key[0], -n_day)
    data_temp = data[(data.visit_date < key[0]) & (data.visit_date > start_date)].copy()
    data_temp['visit_date'] = data_temp['visit_date'].apply(lambda x: diff_of_days(key[0],x))
    data_temp['weight'] = data_temp['visit_date'].apply(lambda x: 0.985**x)
    data_temp['visitors'] = data_temp['visitors'] * data_temp['weight']
    result1 = data_temp.groupby(['air_genre_name', 'dow'], as_index=False)['visitors'].agg({'genre_dow_exp_mean{}'.format(n_day): 'sum'})
    result2 = data_temp.groupby(['air_genre_name', 'dow'], as_index=False)['weight'].agg({'genre_dow_exp_weight_sum{}'.format(n_day): 'sum'})
    result = result1.merge(result2, on=['air_genre_name', 'dow'], how='left')
    result['genre_dow_exp_mean{}'.format(n_day)] = result['genre_dow_exp_mean{}'.format(n_day)]/result['genre_dow_exp_weight_sum{}'.format(n_day)]
    result = left_merge(label, result, on=['air_genre_name', 'dow']).fillna(0)
    return result

In [29]:
def get_first_last_time(label, key, n_day):
    """ Get the first and last date the store is visited.
    Args:
        label: 
        key (tupple): end_date(str), n_day(int)
        n_day: 
    Returns:
        result (pd.DataFrame)
    """
    start_date = date_add_days(key[0], -n_day)
    data_temp = data[(data.visit_date < key[0]) & (data.visit_date > start_date)].copy()
    data_temp = data_temp.sort_values('visit_date')
    result = data_temp.groupby('store_id').agg(
        first_time=('visit_date', lambda x: diff_of_days(key[0],np.min(x))),
        last_time=('visit_date', lambda x: diff_of_days(key[0],np.max(x))))
    result = left_merge(label, result, on=['store_id']).fillna(0)
    return result

In [30]:
def get_air_reserve_feat(label, key, label_end_date):
    """ Get features which utilize air reserve data.
    Arg:
        label:
        key (tupple): end_date(str), n_day(int):
    Returns:
        result (pd.DataFrame)
    """
    label_end_date = date_add_days(key[0], key[1])
    
    air_reserve_temp = air_reserve[(air_reserve.visit_date >= key[0]) &             # key[0] 是'2017-04-23'
                                   (air_reserve.visit_date < label_end_date) &      # label_end_date 是'2017-05-31'
                                   (air_reserve.reserve_date < key[0])].copy()
    air_reserve_temp = air_reserve_temp.merge(air_store,on='store_id',how='left')
    air_reserve_temp['diff_time'] = (pd.to_datetime(air_reserve['visit_datetime'])-pd.to_datetime(air_reserve['reserve_datetime'])).dt.days
    air_reserve_temp = air_reserve_temp.merge(air_store,on='store_id')
    
    
    air_result = air_reserve_temp.groupby(['store_id', 'visit_date']).agg(
        air_reserve_visitors=("reserve_visitors", "sum"),
        air_reserve_count=("reserve_visitors", "count"))
    air_result = air_result.unstack().fillna(0).stack()
    air_result = left_merge(label, air_result, on=['store_id','visit_date']).fillna(0)
    
    air_store_diff_time_mean = air_reserve_temp.groupby(['store_id', 'visit_date']).agg(
        air_store_diff_time_mean=('diff_time', 'mean'))
    air_store_diff_time_mean = left_merge(label, air_store_diff_time_mean, on=['store_id', 'visit_date']).fillna(0)
    
    air_diff_time_mean = air_reserve_temp.groupby(['visit_date']).agg(
        air_diff_time_mean=('diff_time', 'mean'))
    air_diff_time_mean = left_merge(label, air_diff_time_mean, on=['visit_date']).fillna(0)
    
    
    air_date_result = air_reserve_temp.groupby(['visit_date']).agg(
        air_date_visitors=("reserve_visitors", "sum"),
        air_date_count=("reserve_visitors", "count"))
    air_date_result = left_merge(label, air_date_result, on=['visit_date']).fillna(0)
    
    result = pd.concat([air_result, air_date_result,air_store_diff_time_mean, air_diff_time_mean],axis=1)
    return result


def get_hgp_reserve_feat(label, key, label_end_date):
    """ Get features which utilize hgp reserve data.
    Arg:
        label:
        key (tupple): end_date(str), n_day(int):
    Returns:
        result (pd.DataFrame)
    """
    
    hpg_reserve_temp = hpg_reserve[(hpg_reserve.visit_date >= key[0]) & (hpg_reserve.visit_date < label_end_date) & (hpg_reserve.reserve_date < key[0])].copy()
    hpg_reserve_temp['diff_time'] = (pd.to_datetime(hpg_reserve['visit_datetime']) - pd.to_datetime(hpg_reserve['reserve_datetime'])).dt.days
    
    hpg_result = hpg_reserve_temp.groupby(['store_id', 'visit_date']).agg(
        hpg_reserve_visitors=('reserve_visitors', 'sum'),
        hpg_reserve_count=('reserve_visitors', 'count'))
    hpg_result = hpg_result.unstack().fillna(0).stack()
    hpg_result = left_merge(label, hpg_result, on=['store_id', 'visit_date']).fillna(0)
    
    hpg_date_result = hpg_reserve_temp.groupby(['visit_date']).agg(
        hpg_date_visitors=('reserve_visitors', 'sum'),
        hpg_date_count=('reserve_visitors', 'count'))
    hpg_date_result = left_merge(label, hpg_date_result, on=['visit_date']).fillna(0)
    
    hpg_store_diff_time_mean = hpg_reserve_temp.groupby(['store_id', 'visit_date']).agg(
        hpg_store_diff_time_mean=('diff_time','mean'))
    hpg_store_diff_time_mean = left_merge(label, hpg_store_diff_time_mean, on=['store_id', 'visit_date']).fillna(0)
    
    hpg_diff_time_mean = hpg_reserve_temp.groupby(['visit_date']).agg(
        hpg_diff_time_mean=('diff_time','mean'))
    hpg_diff_time_mean = left_merge(label, hpg_diff_time_mean, on=['visit_date']).fillna(0)

    
    result = pd.concat([hpg_result, hpg_date_result, hpg_store_diff_time_mean, hpg_diff_time_mean],axis=1)
    return result

In [31]:
# second feature
def second_feat(result):
    """ Get the rate of the number of visitors some pair of weeks. 
    Arg:
        result (pd.DataFrame):  
        key (tupple): end_date(str), n_day(int):
    Returns:
        result (pd.DataFrame)
    """
    result['store_mean_14_28_rate'] = result['store_mean14']/(result['store_mean28']+0.01)
    result['store_mean_28_56_rate'] = result['store_mean28'] / (result['store_mean56'] + 0.01)
    result['store_mean_56_1000_rate'] = result['store_mean56'] / (result['store_mean1000'] + 0.01)
    result['genre_mean_28_56_rate'] = result['genre_mean28'] / (result['genre_mean56'] + 0.01)
    result['sgenre_mean_56_1000_rate'] = result['genre_mean56'] / (result['genre_mean1000'] + 0.01)
    return result

In [50]:
def make_feats(end_date,n_day):
    t0 = time.time()
    key = end_date,n_day
    print('data key为：{}'.format(key))
    print('add label')
    label, label_end_date = get_label(end_date,n_day)
    print(min(label['visit_date']), max(label['visit_date']))

    print('make feature...')
    result = [label]
    result.append(get_store_visitor_feat(label, key, 1000))        # store features
    """
    result.append(get_store_visitor_feat(label, key, 56))          # store features
    result.append(get_store_visitor_feat(label, key, 28))          # store features
    result.append(get_store_visitor_feat(label, key, 14))          # store features
    
    result.append(get_store_exp_visitor_feat(label, key, 1000))    # store exp features
    
    result.append(get_store_week_feat(label, key, 1000))           # store dow features
    result.append(get_store_week_feat(label, key, 56))             # store dow features
    result.append(get_store_week_feat(label, key, 28))             # store dow features
    result.append(get_store_week_feat(label, key, 14))             # store dow features
    
    result.append(get_store_week_diff_feat(label, key, 1000))      # store dow diff features
    result.append(get_store_week_diff_feat(label, key, 58))        # store dow diff features
    
    result.append(get_store_all_week_feat(label, key, 1000))       # store all week feat
    result.append(get_store_week_exp_feat(label, key, 1000))       # store dow exp feat
    result.append(get_store_holiday_feat(label, key, 1000))        # store holiday feat

    result.append(get_genre_visitor_feat(label, key, 1000))         # genre feature
    result.append(get_genre_visitor_feat(label, key, 56))           # genre feature
    result.append(get_genre_visitor_feat(label, key, 28))           # genre feature
    result.append(get_genre_exp_visitor_feat(label, key, 1000))     # genre feature
    
    result.append(get_genre_week_feat(label, key, 1000))            # genre dow feature
    result.append(get_genre_week_feat(label, key, 56))              # genre dow feature
    result.append(get_genre_week_feat(label, key, 28))              # genre dow feature
    result.append(get_genre_week_exp_feat(label, key, 1000))        # genre dow exp feature

    result.append(get_air_reserve_feat(label,key, label_end_date))                  # air_reserve
    result.append(get_hgp_reserve_feat(label,key, label_end_date))
    result.append(get_first_last_time(label,key,1000)) """ # first time and last time

    result.append(label)

    print('merge...')
    result = concat(result)

    #result = second_feat(result)

    print('data shape：{}'.format(result.shape))
    print('spending {}s'.format(time.time() - t0))
    return result

39 が来ているのは、学習用データの最後40日をオフラインの評価用データセットにしているからかもしれない。

In [51]:
train_feat = pd.DataFrame()
start_date = '2017-03-12'



#for i in range(58):
for i in range(3):
    train_feat_sub = make_feats(date_add_days(start_date, i*(-7)),39)
    train_feat = pd.concat([train_feat,train_feat_sub], axis=0)

#for i in range(1,6):
for i in range(1,2):
    train_feat_sub = make_feats(date_add_days(start_date,i*(7)),42-(i*7))
    train_feat = pd.concat([train_feat,train_feat_sub], axis=0)

data key为：('2017-03-12', 39)
add label
2017-03-12 2017-04-19
make feature...
merge...
data shape：(27728, 31)
spending 2.362940788269043s
data key为：('2017-03-05', 39)
add label
2017-03-05 2017-04-12
make feature...
merge...
data shape：(27705, 31)
spending 2.3641679286956787s
data key为：('2017-02-26', 39)
add label
2017-02-26 2017-04-05
make feature...
merge...
data shape：(27745, 31)
spending 2.3809900283813477s
data key为：('2017-03-19', 35)
add label
2017-03-19 2017-04-22
make feature...
merge...
data shape：(24969, 31)
spending 2.1525650024414062s


In [40]:
max(train_feat['visit_date'])

'2017-04-22'

In [41]:
min(train_feat['visit_date'])

'2017-02-26'

In [56]:
train_feat[(train_feat["store_id"]=="air_ba937bf13d40fb24") & (train_feat["visit_date"] == '2017-03-12')]

,store_id,visit_date,visitors,id,dow,air_genre_name,air_area_name,latitude,longitude,air_area_name0,...,ahead_holiday2_1,ahead_holiday_-1,ahead_holiday2_-1,store_min1000,store_mean1000,store_median1000,store_max1000,store_count1000,store_std1000,store_skew1000
0,air_ba937bf13d40fb24,2017-03-12,2.484907,air_ba937bf13d40fb24_2017-03-12,6,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7,...,1,0,0,0.693147,3.054871,3.156774,4.127134,352.0,0.585452,-1.125951
6,air_ba937bf13d40fb24,2017-03-12,2.484907,air_ba937bf13d40fb24_2017-03-12,6,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7,...,1,0,0,0.693147,3.058702,3.178054,4.127134,346.0,0.584079,-1.144632
12,air_ba937bf13d40fb24,2017-03-12,2.484907,air_ba937bf13d40fb24_2017-03-12,6,4,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,7,...,1,0,0,0.693147,3.063912,3.178054,4.127134,340.0,0.583355,-1.179524
